In [1]:
import pickle
import os
import glob
import torch
import numpy as np
import pandas as pd
import scipy.io
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from scipy import signal
from scipy import stats
import seaborn as sns
import statsmodels.api as sm
from pyjaspar import jaspardb

In [2]:
# jupyter notebook display setting for all data structures
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
np.set_printoptions(threshold=np.inf)

In [3]:
# define the path to the data directory
data_file = '/project2/xinhe/waqaas/DNA-breathing/data/Chipseq_data/seq_breathing_feat.pkl'

In [4]:
with open(data_file, 'rb') as f:
    panset = pickle.load(f)

In [5]:
for part in panset.keys():
    partition = panset[part]
    for seq_id in partition.keys():
        sample = partition[seq_id]
        # remove the first 150 and last 150 bp from seq, coord, coord_sq, flip
        sample['seq'] = sample['seq'][150:-150]
        sample['coord'] = sample['coord'][150:-150]
        sample['coord_sq'] = sample['coord_sq'][150:-150]
        sample['flip'] = sample['flip'][150:-150]

In [6]:
for part in panset.keys():
    partition = panset[part]
    for seq_id in partition.keys():
        sample = partition[seq_id]
        print(len(sample['seq']))
        print(len(sample['coord']))
        print(len(sample['coord_sq']))
        print(len(sample['flip']))
        break
    break

200
200
200
200


In [7]:
panset_df = pd.DataFrame(columns=['seq_id', 'seq', 'coord', 'coord_sq', 'flip', 'label'])

In [8]:
flat_data = []
for part in panset.keys():
    for seq_id, data in panset[part].items():
        row = {'part': part, 'seq_id': seq_id, **data}
        flat_data.append(row)

# Convert to DataFrame
df = pd.DataFrame(flat_data)

# del column part
del df['part']
#turn seq_id into index
df.set_index('seq_id', inplace=True)

In [9]:
# preview the data
df.head()

,seq,coord,coord_sq,flip,label
seq_id,,,,,
81,CCCCTGAGCTAGCCATGCTCTGACAGTCTCAGTTGCACACACGAGC...,"[0.157, 0.151, 0.1509, 0.154, 0.1974, 0.1612, ...","[0.1185, 0.1127, 0.1125, 0.1145, 0.1418, 0.122...","[0.0615, 0.0579, 0.0577, 0.0592, 0.0698, 0.063...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
146,TCTCGGTCAGGAACGGTCCCGGGCCTCCCGCCCGCCTCCCTCCAGC...,"[0.2238, 0.1714, 0.208, 0.1448, 0.139, 0.1544,...","[0.1728, 0.1373, 0.1551, 0.1037, 0.0981, 0.115...","[0.0888, 0.0709, 0.0782, 0.0522, 0.0483, 0.059...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
147,CCGCCCGCGCCTCCGGGTCCTAACGCCGCCGCTCGCCCTCCACTGC...,"[0.1535, 0.131, 0.1242, 0.1308, 0.1372, 0.1257...","[0.1119, 0.0843, 0.0758, 0.0829, 0.09, 0.0762,...","[0.0601, 0.0436, 0.0382, 0.0438, 0.049, 0.0401...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
449,CAAAGTGCTAAGTATGGTAGATTGCAAACATAAGTGGCCACATAAT...,"[0.1735, 0.2203, 0.2314, 0.22, 0.163, 0.1958, ...","[0.143, 0.171, 0.1812, 0.1683, 0.1258, 0.1401,...","[0.0717, 0.0845, 0.0923, 0.0849, 0.0647, 0.068...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1178,AGCTGACTCATGTTGGGGGCAGCCCACTCACAGTGGCCCTGACCCA...,"[0.1938, 0.1433, 0.1429, 0.1941, 0.1611, 0.210...","[0.1358, 0.0995, 0.0986, 0.1347, 0.1207, 0.157...","[0.0676, 0.0512, 0.051, 0.0675, 0.0639, 0.08, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [10]:
# define path to the TF index
tf_index_file = '/project2/xinhe/waqaas/DNA-breathing/data/Chipseq_data/group_index.pt'

In [11]:
# load the TF index
tf_index = torch.load(tf_index_file)

In [12]:
# print the TF index
print(tf_index)

{'HNF4A': [85], 'JUND': [60, 86], 'NFYB': [22, 63], 'FOXA1': [82, 128], 'SP1': [31], 'EBF1': [14], 'TAF1': [3, 71, 92], 'BCL3': [10], 'CHD2': [56], 'ELF1': [16, 79, 105], 'POLR2A': [1, 25, 38, 41, 64, 88, 98, 114], 'BRCA1': [54], 'ZNF143': [76], 'BHLHE40': [12], 'FOSL1': [107], 'NR2F2': [112], 'GABPA': [19, 59, 84, 108], 'THAP1': [122], 'GATA3': [127, 129], 'BATF': [8], 'BCLAF1': [11, 101], 'IRF4': [20], 'RAD21': [27, 48, 66, 89, 99, 115], 'TCF7L2': [73], 'REST': [28, 49, 67, 116], 'TBP': [72], 'E2F6': [103], 'RUNX3': [29], 'SPI1': [32, 40, 118], 'TAF7': [120], 'USF2': [75], 'SMC3': [69], 'PBX3': [24], 'POU2F2': [26, 39], 'BCL11A': [9], 'EP300': [0, 17, 58, 80], 'ATF3': [100], 'EGR1': [15, 104], 'ETS1': [18, 106], 'MAX': [47, 62, 87, 110], 'TRIM28': [123], 'CEBPB': [55, 77, 95], 'SIX5': [2, 30, 117], 'TFAP2C': [74], 'FOSL2': [81], 'YY1': [4, 36, 42, 53, 94, 125], 'FOXA2': [83], 'USF1': [35, 52, 93, 124], 'PML': [113], 'CTCF': [5, 6, 7, 13, 43, 44, 45, 46, 57, 78, 96, 102], 'TEAD4': [51

In [13]:
# make a dictionary called motifs using keys from tf_index
motifs = {}
for key in tf_index.keys():
    motifs[key] = None

In [14]:
# create jaspar object
jdb_obj = jaspardb(release='JASPAR2022')

In [15]:
for TF in motifs.keys():
    # fetch the motif
    motif = jdb_obj.fetch_motifs(
        tf_name = TF,
        species = '9606'
    )
    # store the motif in the dictionary
    motifs[TF] = motif

/home/waqaas/anaconda3/lib/python3.10/site-packages/pyjaspar/__init__.py:755: BiopythonWarning: Zero motifs returned with current select critera
  warnings.warn("Zero motifs returned with current select critera",


In [16]:
for TF in motifs.keys():
    # fetch the motif
    motif_group = motifs[TF]
    # continue if TF is empty
    if motif_group is None:
        continue
    # print the motif
    for motif in motif_group:
        # turn motif into a string
        motif_str = str(motif)
        matrix_id = motif_str.split('\n')[1].split('\t')[1]
        # wget the motif in meme format
        os.system(f"wget -O chipseq_motifs/{TF}/{matrix_id}.meme http://jaspar.genereg.net/api/v1/matrix/{matrix_id}.meme")
        # pfm = pd.DataFrame(motif.counts)
        # pfm = pfm.T.to_string(index=False, header=False)
        # # save the motif
        # motif_saver(pfm, matrix_id, TF, f"{matrix_id}.pfm")

--2024-04-12 22:37:15--  http://jaspar.genereg.net/api/v1/matrix/MA1494.1.meme
Resolving jaspar.genereg.net (jaspar.genereg.net)... 193.60.222.202
Connecting to jaspar.genereg.net (jaspar.genereg.net)|193.60.222.202|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://jaspar.elixir.no/api/v1/matrix/MA1494.1.meme [following]
--2024-04-12 22:37:15--  https://jaspar.elixir.no/api/v1/matrix/MA1494.1.meme
Resolving jaspar.elixir.no (jaspar.elixir.no)... 158.39.201.109
Connecting to jaspar.elixir.no (jaspar.elixir.no)|158.39.201.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 834 [text/meme]
Saving to: ‘chipseq_motifs/HNF4A/MA1494.1.meme’

     0K                                                       100% 11.1M=0s

2024-04-12 22:37:17 (11.1 MB/s) - ‘chipseq_motifs/HNF4A/MA1494.1.meme’ saved [834/834]

--2024-04-12 22:37:18--  http://jaspar.genereg.net/api/v1/matrix/MA0114.4.meme
Resolving jaspar.genereg.net (jaspar.ge

In [17]:
# turn df into a FASTA
def df_to_fasta(df, fasta_file):
    # open the fasta file
    with open(fasta_file, 'w') as f:
        # iterate through the rows of the df
        for index, row in df.iterrows():
            # write the header
            f.write(f">{index}\n")
            # write the sequence
            f.write(f"{row['seq']}\n")

In [18]:
# save the FASTA if file doesn't exist already
if not os.path.isfile('../../../TF_fastas/chipseq.fasta'):
    df_to_fasta(df, '../../../TF_fastas/chipseq.fasta')

In [28]:
# create a directory called chipseq_results
os.system('mkdir chipseq_results')
# get every .meme file in the chipseq_motifs directory as well as its path
meme_files = glob.glob('chipseq_motifs/*/*.meme')

In [29]:
# give chipseq_results the same directory structure as chipseq_motifs
for meme_file in meme_files:
    # get the path of the meme file and remove the .meme extension
    meme_path = meme_file.replace('.meme', '')
    # remove chipseq_motifs from the path
    meme_path = meme_path.replace('chipseq_motifs/', '')
    # create the same path in chipseq_results
    os.system(f"mkdir -p chipseq_results/{meme_path}")

In [30]:
# get current working directory
cwd = os.getcwd()
print(cwd)

/project2/xinhe/waqaas/DNA-breathing/src/fasta


In [31]:
# change working directory to chipseq_results
os.chdir('chipseq_results')
# get every .meme file in the chipseq_motifs directory as well as its path
meme_files = glob.glob('../chipseq_motifs/*/*.meme')

In [32]:
for meme_file in meme_files:
    print(meme_file.replace('.meme', ''))
    break

../chipseq_motifs/TCF12/MA1648.1


In [33]:
# navigate to the chipseq_results directory and run 'fimo --no-pgc chipseq_motifs/*/*.meme ../../../TF_fastas/chipseq.fasta'
for meme_file in meme_files:
    os.chdir(f'{(meme_file.replace(".meme", "")).replace("chipseq_motifs/", "chipseq_results/")}')
    os.system(f'fimo --no-pgc --thresh 0.001 --max-stored-scores 400000 ../../{meme_file} ../../../../../../TF_fastas/chipseq.fasta')
    os.chdir('../../')

Using motif +MA1648.1 of width 11.
Using motif -MA1648.1 of width 11.
Motif matches with p-value >= 0.00043 have been dropped to reclaim memory.
Finding best site passing the output threshold in each of the 886625 sequences.
Found a best site in 278716 sequences.
Computing q-values.
# Computing q-values.
#   Estimating pi_0 from a uniformly sampled set of 10000 p-values.
#   Estimating pi_0.
# Minimal pi_zero = 0.965935
#   Estimated pi_0=0.973933
Using motif +MA0473.3 of width 14.
Using motif -MA0473.3 of width 14.
Motif matches with p-value >= 0.00041 have been dropped to reclaim memory.
Finding best site passing the output threshold in each of the 886625 sequences.
Found a best site in 305486 sequences.
Computing q-values.
# Computing q-values.
#   Estimating pi_0 from a uniformly sampled set of 10000 p-values.
#   Estimating pi_0.
# Minimal pi_zero = 0.973529
#   Estimated pi_0=0.980957
Using motif +MA0058.3 of width 10.
Using motif -MA0058.3 of width 10.
Finding best site passing 